In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)

In [2]:
with open('/datasets/stop_words_en.txt') as f:
    stop_words = f.read().splitlines()

In [3]:
import operator 

filtered_words = wiki.map(lambda words: [word.lower() for word in words])\
                        .map(lambda words: [word for word in words if word not in stop_words]).persist()                     

collocations = filtered_words.flatMap(lambda xs: ('_'.join(x) for x in zip(xs, xs[1:])))


In [4]:
filtered_collocations = collocations.map(lambda x: (x, 1))\
                                    .reduceByKey(lambda x, y: x + y)\
                                    .filter(lambda x: x[1]>=500)

In [ ]:
from math import log
def npmi(bigram_pairs):
    bigram, pab = bigram_pairs
    a, b = bigram.split('_', 1)
    pa = p_dict[a]                                                                                 
    pb = p_dict[b]
    pmi = log(pab/(pa * pb))
    npmi = pmi/-log(pab)
    return (bigram, npmi)

In [ ]:
num_words = filtered_words.flatMap(lambda x: x).count()
#num_words = filtered_words.map(len).reduce(operator.add)
p_words = filtered_words.flatMap(lambda x: x).map(lambda x: (x, 1))\
                                            .reduceByKey(operator.add)\
                                            .filter(lambda x: x[1] >= 500)\
                                            .map(lambda x: (x[0], float(x[1])/num_words)).persist()
p_dict = dict(p_words.collect()) 

In [ ]:
count_pairs = collocations.count()
filtered_collocations = filtered_collocations.map(lambda x: (x[0], float(x[1]) / count_pairs))

In [ ]:
npmi_pairs = filtered_collocations.map(npmi)

In [ ]:
result = npmi_pairs.sortBy(lambda a: -a[1]).take(39)
#result = npmi_pairs.takeOrdered(39, key = lambda x: -x[1])

In [ ]:
for e in result:
    print e[0]

los_angeles
external_links
united_states
prime_minister
san_francisco
et_al
new_york
supreme_court
19th_century
20th_century
references_external
soviet_union
air_force
baseball_player
university_press
roman_catholic
united_kingdom
references_reading
notes_references
award_best
north_america
new_zealand
civil_war
catholic_church
world_war
war_ii
south_africa
took_place
roman_empire
united_nations
american_singer-songwriter
high_school
american_actor
american_actress
american_baseball
york_city
american_football
years_later
north_american
